# AgentAction and AgentFinish

In [1]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain_chroma import Chroma

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama

from langchain.tools.retriever import create_retriever_tool
from langchain.tools import tool

from langchain import hub

from platform import python_version

from langchain.agents import create_tool_calling_agent, AgentExecutor

In [2]:
wikipedia_tool = WikipediaQueryRun(api_wrapper = WikipediaAPIWrapper())

In [3]:
vectorstore = Chroma(persist_directory = "./intro-to-ds-lectures", 
                     embedding_function = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2"))

retriever = vectorstore.as_retriever(search_type = 'mmr', 
                                     search_kwargs = {'k':3, 
                                                      'lambda_mult':0.7})

retriever_tool = create_retriever_tool(retriever = retriever, 
                                       name = "Introduction-to-Data-and-Data-Science-Course-Lectures", 
                                       description = '''For any questions regarding 
                                       the Introduction to Data and Data Science course, you must use this tool.''')

In [4]:
@tool
def get_python_version() -> str:
    ''' Useful for questions regarding the version of Python currently used. '''
    return python_version()

In [5]:
tools = [wikipedia_tool, retriever_tool, get_python_version]

In [6]:
chat = ChatOllama(model="llama3.2", 
                  temperature=0,
                  seed=365)

In [7]:
chat_prompt_template = hub.pull("hwchase17/openai-tools-agent")

c:\Users\Eda AYDIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [8]:
agent = create_tool_calling_agent(llm = chat, 
                                  tools = tools, 
                                  prompt = chat_prompt_template)

In [12]:
agent.invoke({"input":"Could you tell me the version of Python I'm currently using?", 
              "intermediate_steps":response["intermediate_steps"]})

AgentFinish(return_values={'output': 'The current version of Python you are using is 3.12.6.'}, log='The current version of Python you are using is 3.12.6.')

In [10]:
agent_executor = AgentExecutor(agent = agent, 
                               tools = tools, 
                               verbose = True, 
                               return_intermediate_steps = True)

In [11]:
response = agent_executor.invoke({"input":"Could you tell me the version of Python I'm currently using?"})



> Entering new AgentExecutor chain...

Invoking: `get_python_version` with `{}`


3.12.6The current version of Python is 3.12.6.

> Finished chain.


In [ ]:
response

{'input': "Could you tell me the version of Python I'm currently using?",
 'output': 'The version of Python currently being used is 3.10.10.',
 'intermediate_steps': [(ToolAgentAction(tool='get_python_version', tool_input={}, log='\nInvoking: `get_python_version` with `{}`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_lo6ih5XF62vPNrCxcMkKZIqh', 'function': {'arguments': '{}', 'name': 'get_python_version'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-9464a78d-3eb1-4b38-a18e-53537d2dc453', tool_calls=[{'name': 'get_python_version', 'args': {}, 'id': 'call_lo6ih5XF62vPNrCxcMkKZIqh'}], tool_call_chunks=[{'name': 'get_python_version', 'args': '{}', 'id': 'call_lo6ih5XF62vPNrCxcMkKZIqh', 'index': 0}])], tool_call_id='call_lo6ih5XF62vPNrCxcMkKZIqh'),
   '3.10.10')]}